In [ ]:
import numpy as np

In [ ]:
def mse(y, y_pred):
    return np.mean((y - y_pred) ** 2)

def infer(x, coefficients):
    return np.dot(x, coefficients)

# Minimize the OLS function using the gradient descent method
def minimize_ols(x, y):
    # Initialize the coefficients with zeros
    coefficients = np.zeros(x.shape[1], np.dtype(np.float64))
    coef_hist = [coefficients.copy()]
    mse_hist = [mse(y, infer(x, coefficients))]

    # Set the learning rate and number of iterations
    learning_rate = np.array(0.002, dtype=np.float64)
    num_iterations = 100000

    # Perform gradient descent
    for i in range(num_iterations):
        # Calculate the predicted values
        y_pred = infer(x, coefficients)
        print(f"Coefs ({coefficients.shape}): {coefficients}")

        # Calculate the error
        error = y_pred - y

        # Update the coefficients
        coef_gradients = np.dot(x.T, error) / len(x)
        coef_update = learning_rate * coef_gradients

        # Check if coef_update is too small to break the loop
        if np.all(coef_update > -np.inf):
            print(f"Update from error:\n ({coef_update.shape}): {coef_update}")
            coefficients -= coef_update
            coef_hist.append(coefficients.copy())
            mse_val = mse(y, infer(x, coefficients))
            print(f"MSE: {mse_val}")
            mse_hist.append(mse_val)
        else:
            print(f"Update too small")
            break

    return coefficients, coef_hist, mse_hist

In [ ]:
# Minimize the OLS function using the analytical equation
def minimize_ols2(x, y):
    # Calculate the coefficients
    coefficients = np.dot(np.linalg.inv(np.dot(x.T, x)), np.dot(x.T, y))
    return coefficients


In [ ]:
x = np.linspace(0, 1, 100)
y = 3 * x - 1 + np.random.normal(loc=0, scale=0.5, size=len(x))
X = x[:, None].astype(np.float64)
Y = y[:].astype(np.float64)
X_ext = np.hstack((np.ones((len(X), 1)), X)) # To include the bias term

In [ ]:
ax, ax_hist, mse_hist = minimize_ols(X_ext, Y)
print("Params: ", ax)

In [ ]:
ax2 = minimize_ols2(X_ext, Y)
print("Params: ", ax2)
print("MSE: ", mse(Y, infer(X_ext, ax2)))

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation

In [ ]:
# To obtain a smooth animation, we will use a logarithmic scale for the last iterations.
# The first values of the log space are too closed so a linear space is used instead, and the last values are logarithmically spaced.
# This is equivalent to the np.unique(logspace) function but suppousedly faster.
logspace = np.logspace(start=0, stop=np.log10(len(ax_hist)-1), num=1000, dtype=np.int64)
logspace_dist1 = np.hstack((False, (logspace[1:] - logspace[:-1]) > 0))
idx_logspace_part = logspace[logspace_dist1]
last_linear_idx = idx_logspace_part[0]
idx_linspace_part = np.arange(last_linear_idx)
idx = np.hstack((idx_linspace_part, idx_logspace_part))

animation_fig = plt.figure()
animation_ax = plt.axes(xlim=(0, 1), ylim=(-5, 5))
animation_line, = animation_ax.plot([], [], lw=2)
scat = animation_ax.scatter(x, y, c='r', s=10)
normal_method_line, = animation_ax.plot(x, ax2[1] * x + ax2[0], lw=2, c='g')
animation_bias_text = animation_ax.text(0.02, 0.95, '', transform=animation_ax.transAxes)
animation_coef_text = animation_ax.text(0.02, 0.90, '', transform=animation_ax.transAxes)
animation_mse_text = animation_ax.text(0.02, 0.85, '', transform=animation_ax.transAxes)
animation_it_text = animation_ax.text(0.02, 0.80, '', transform=animation_ax.transAxes)

def init():
    animation_line.set_data([], [])
    animation_bias_text.set_text('')
    animation_coef_text.set_text('')
    animation_mse_text.set_text('')
    animation_it_text.set_text('')
    return animation_line, animation_bias_text, animation_coef_text, animation_mse_text

def animate(idx_i):
    i = idx[idx_i]
    animation_line.set_data(x, ax_hist[i][1] * x + ax_hist[i][0])
    animation_bias_text.set_text(f'Bias = {ax_hist[i][0]:.2f}')
    animation_coef_text.set_text(f'Coefficients = {ax_hist[i][1]:.2f}')
    animation_mse_text.set_text(f'MSE = {mse_hist[i]:.4f}')
    animation_it_text.set_text(f'Iter = {i}')
    return animation_line, animation_bias_text, animation_coef_text, animation_mse_text

anim = animation.FuncAnimation(animation_fig, animate, init_func=init,
                                 frames=len(idx), interval=100, blit=True)
anim.save('animation.gif', writer='imagemagick', fps=30)
plt.show()